In [1]:
import csv
import chardet
import logging
import os
import pandas as pd
import sqlite3
import zipfile
from datetime import datetime


In [2]:
def configurar_log(caminho_arquivo):
    logging.basicConfig(
        filename=caminho_arquivo,
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )



def registrar_log(mensagem, nivel='info'):
    print(mensagem)
    if nivel == 'info':
        logging.info(mensagem)
    elif nivel == 'warning':
        logging.warning(mensagem)
    elif nivel == 'error':
        logging.error(mensagem)
    elif nivel == 'debug':
        logging.debug(mensagem)
    else:
        logging.info(f"Nível desconhecido: {mensagem}")



def criar_banco_dados(caminho_arquivo):
    try:
        conexao = sqlite3.connect(caminho_arquivo)
        registrar_log(f"Banco de dados '{caminho_arquivo}' criado com sucesso.", nivel='info')
        return caminho_arquivo
    except sqlite3.Error as erro:
        registrar_log(f"Erro ao criar o banco de dados: {erro}", nivel='error')
    finally:
        if conexao:
            conexao.close()



def descompactar_arquivos(pasta_origem, pasta_destino=None):
    if not os.path.isdir(pasta_origem):
        registrar_log(f"Erro: A pasta de origem '{pasta_origem}' não existe ou não é um diretório.", nivel='error')
        return

    if pasta_destino and not os.path.exists(pasta_destino):
        try:
            os.makedirs(pasta_destino)
            registrar_log(f"Criada pasta de destino: {pasta_destino}", nivel='info')
        except Exception as erro:
            registrar_log(f"Erro ao criar pasta de destino '{pasta_destino}': {erro}", nivel='error')
            return

    registrar_log(f"Descompactando arquivos da pasta '{pasta_origem}'...", nivel='info')
    
    for nome_arquivo in os.listdir(pasta_origem):
        caminho_arquivo = os.path.join(pasta_origem, nome_arquivo)
        if zipfile.is_zipfile(caminho_arquivo):
            try:
                with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
                    destino = pasta_destino if pasta_destino else pasta_origem
                    registrar_log(f"Descompactando '{nome_arquivo}' para '{destino}'", nivel='info')
                    zip_ref.extractall(destino)
            except zipfile.BadZipFile:
                registrar_log(f"Arquivo compactado corrompido ou inválido: {nome_arquivo}", nivel='error')
            except PermissionError:
                registrar_log(f"Permissão negada ao acessar: {nome_arquivo}", nivel='error')
            except Exception as erro:
                registrar_log(f"Erro ao descompactar o arquivo '{nome_arquivo}': {erro}", nivel='error')
        else:
            registrar_log(f"Ignorado (não é um arquivo compactado): {nome_arquivo}", nivel='info')

    registrar_log(f"Descompactação de arquivos da pasta '{pasta_origem}' concluído.", nivel='info')


def detectar_delimitador(caminho_arquivo, encoding, num_linhas=5):
    with open(caminho_arquivo, 'r', encoding=encoding) as f:
        amostra = ''.join([f.readline() for _ in range(num_linhas)])
        sniffer = csv.Sniffer()
        try:
            dialect = sniffer.sniff(amostra)
            return dialect.delimiter
        except csv.Error:
            return ','  # padrão



def detectar_encoding(caminho_arquivo, num_bytes=10000):
    with open(caminho_arquivo, 'rb') as f:
        amostra = f.read(num_bytes)
        resultado = chardet.detect(amostra)
        return resultado['encoding'] if resultado['encoding'] else 'utf-8'



def excluir_arquivo(caminho_arquivo):
    if os.path.exists(caminho_arquivo):
        try:
            os.remove(caminho_arquivo)
            registrar_log(f"Arquivo '{caminho_arquivo}' excluído com sucesso.", nivel='info')
        except Exception as erro:
            registrar_log(f"Erro ao excluir o arquivo: {erro}", nivel='error')
    else:
        registrar_log(f"O arquivo '{caminho_arquivo}' não existe.", nivel='error')



def executar_script_sql(nome_banco, caminho_script):
    try:
        conexao = sqlite3.connect(nome_banco)
        cursor = conexao.cursor()

        with open(caminho_script, 'r', encoding='utf-8') as arquivo_sql:
            script_sql = arquivo_sql.read()

        cursor.executescript(script_sql)

        conexao.commit()
        registrar_log(f"Script SQL '{caminho_script}' executado com sucesso.", nivel='info')
    except Exception as erro:
        registrar_log(f"Erro ao executar o script SQL: {erro}", nivel='error')
    finally:
        coenxao.close()



def importar_arquivos_para_banco_dados(pasta_origem, nome_banco, conjunto_dados):
    if not os.path.isdir(pasta_origem):
        registrar_log(f"Erro: A pasta de origem '{pasta_origem}' não existe.", nivel='error')
        return

    try:
        conn = sqlite3.connect(nome_banco)
    except Exception as erro:
        registrar_log(f"Erro ao conectar ao banco de dados: {erro}", nivel='error')
        return

    for nome_arquivo in os.listdir(pasta_origem):
        caminho_arquivo = os.path.join(pasta_origem, nome_arquivo)
        if os.path.isfile(caminho_arquivo):
            try:

                raiz_arquivo, extensao_arquivo = os.path.splitext(nome_arquivo)
                if extensao_arquivo in (".EMPRECSV", ".ESTABELE", ".SOCIOCSV"):
                    encoding = 'latin-1'
                else:
                    encoding = detectar_encoding(caminho_arquivo)
                delimitador = detectar_delimitador(caminho_arquivo, encoding)
                
                registrar_log(f"Processando arquivo '{caminho_arquivo}', com encoding '{encoding}', delimitador '{delimitador}'...", nivel='info')
                
                with open(caminho_arquivo, 'r', encoding=encoding) as f:
                    primeira_linha = f.readline()
                    if nome_arquivo.endswith('.csv') or nome_arquivo.endswith('.txt'):
                        tem_cabecalho = csv.Sniffer().has_header(primeira_linha)
                    else:
                        tem_cabecalho = None

                df = pd.read_csv(
                    caminho_arquivo,
                    delimiter=delimitador,
                    header=0 if tem_cabecalho else None,
                    encoding=encoding,
                    low_memory=False
                )

                if not tem_cabecalho:
                    df.columns = [f'coluna_{i+1}' for i in range(df.shape[1])]

                # Normalização: remove espaços, converte para string e trata nulos
                df = df.map(lambda x: str(x).strip() if pd.notnull(x) else None)

                df.to_sql(conjunto_dados + '_' + nome_arquivo, conn, if_exists='append', index=False)
                registrar_log(f"Arquivo '{caminho_arquivo}' importado com sucesso.", nivel='info')
                excluir_arquivo(caminho_arquivo)

            except Exception as erro:
                registrar_log(f"Erro ao importar o arquivo '{nome_arquivo}': {erro}", nivel='error')

    conn.close()
    
    registrar_log(f"Importação de arquivos da pasta '{pasta_origem}' concluída.", nivel='info')




In [3]:
pasta_projeto = r'C:\Stage\Projects\TCC_Univesp'
pasta_dados = pasta_projeto + '\\dados'

configurar_log(pasta_projeto + '\\TCC_Univesp.log')
registrar_log('Início do processo.')

banco_dados = criar_banco_dados(pasta_dados + '\\TCC_Univesp.db')
banco_dados = pasta_dados + '\\TCC_Univesp.db'


Início do processo.
Banco de dados 'C:\Stage\Projects\TCC_Univesp\dados\TCC_Univesp.db' criado com sucesso.


In [4]:
conjunto_dados = 'RFB'

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

Descompactando arquivos da pasta 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\raw'...
Descompactando 'Cnaes.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas0.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas1.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas2.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas3.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas4.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas5.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas6.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas7.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas8.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\RFB\CNPJ\csv'
Descompactando 'Empresas9.zip' para 'C:\Stage\P

In [5]:
conjunto_dados = 'TSE'

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\TSE\2020'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\TSE\2022'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\TSE\2024'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

Descompactando arquivos da pasta 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\raw'...
Descompactando 'consulta_cand_2018.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv'
Descompactando 'consulta_cand_complementar_2018.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv'
Descompactando 'prestacao_de_contas_eleitorais_candidatos_2018.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv'
Descompactação de arquivos da pasta 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\raw' concluído.
Processando arquivo 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv\consulta_cand_2018_AC.csv', com encoding 'ISO-8859-1', delimitador ';'...
Arquivo 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv\consulta_cand_2018_AC.csv' importado com sucesso.
Arquivo 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv\consulta_cand_2018_AC.csv' excluído com sucesso.
Processando arquivo 'C:\Stage\Projects\TCC_Univesp\dados\TSE\2018\csv\consulta_cand_2018_AL.csv', com encoding 'ISO-8859-1', de

In [6]:
conjunto_dados = 'CGU'

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\CGU\Contratos'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

pasta_processamento = r'C:\Stage\Projects\TCC_Univesp\dados\CGU\Codevasf'
descompactar_arquivos(pasta_processamento + '\\raw', pasta_processamento + '\\csv')
importar_arquivos_para_banco_dados(pasta_processamento + '\\csv', banco_dados, conjunto_dados)

Descompactando arquivos da pasta 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\raw'...
Ignorado (não é um arquivo compactado): EmendasParlamentares
Descompactando 'EmendasParlamentares.zip' para 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\csv'
Descompactação de arquivos da pasta 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\raw' concluído.
Processando arquivo 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\csv\EmendasParlamentares.csv', com encoding 'ISO-8859-1', delimitador ';'...
Arquivo 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\csv\EmendasParlamentares.csv' importado com sucesso.
Arquivo 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\csv\EmendasParlamentares.csv' excluído com sucesso.
Processando arquivo 'C:\Stage\Projects\TCC_Univesp\dados\CGU\EmendasParlamentares\csv\EmendasParlamentares_Convenios.csv', com encoding 'ISO-8859-1', delimitador ';'...
Arquivo 'C:\Stage\Projects\TCC_Univ

In [7]:
#script_sql = pasta_dados + '\\Normalizar_Dados.sql'
#executar_script_sql(banco_dados, script_sql)